In [ ]:
""" 
Functions for biased paramter search.
"""

import os
import pickle
import numpy as np

from numba import jit
from time import time
from tqdm import tqdm
from datetime import datetime
from multiprocessing import Pool
from scipy.integrate import odeint
from scipy.signal import periodogram

import warnings
warnings.filterwarnings("ignore")

error_message = 'Excess work done on this call (perhaps wrong Dfun type).'

In [ ]:
@jit(nopython=True)
def DvD(S, t, k, K):
    """
    The function of the system for scipy.integrate.odeint.
    
    Parameters
    --------------
    S : array
    Condition of substrates 
    t : array
    A sequence of time points.
    k : array
    Rate constants.
    K: array
    MM constants.
    
    Returns
    ----------
    Sintg : array
    The change of S.
    """
    
    Sintg = np.empty(6)
    Sa_00, Sa_01, Sa_10, Sb_00, Sb_01, Sb_10 = S
    
    E = 20./(1 + Sa_00/K[0] + Sa_00/K[1] + Sa_01/K[2]   + Sa_10/K[3]
                  + Sb_00/K[8] + Sb_00/K[9] + Sb_01/K[10] + Sb_10/K[11])
    F = 20./(1 + Sa_01/K[4]   + Sa_10/K[5]   + (1000.-Sa_00-Sa_01-Sa_10)/K[6]   + (1000.-Sa_00-Sa_01-Sa_10)/K[7]
                  + Sb_01/K[12] + Sb_10/K[13] + (1000.-Sb_00-Sb_01-Sb_10)/K[14] + (1000.-Sb_00-Sb_01-Sb_10)/K[15])
             
    Sintg[0] = - k[0]*E*Sa_00/K[0] - k[1]*E*Sa_00/K[1] + k[4]*F*Sa_01/K[4] + k[5]*F*Sa_10/K[5]
    Sintg[1] = - k[4]*F*Sa_01/K[4] - k[2]*E*Sa_01/K[2] + k[0]*E*Sa_00/K[0] + k[6]*F*(1000.-Sa_00-Sa_01-Sa_10)/K[6]
    Sintg[2] = - k[5]*F*Sa_10/K[5] - k[3]*E*Sa_10/K[3] + k[1]*E*Sa_00/K[1] + k[7]*F*(1000.-Sa_00-Sa_01-Sa_10)/K[7]
    Sintg[3] = - k[8]*E*Sb_00/K[8] - k[9]*E*Sb_00/K[9] + k[12]*F*Sb_01/K[12] + k[13]*F*Sb_10/K[13]
    Sintg[4] = - k[12]*F*Sb_01/K[12] - k[10]*E*Sb_01/K[10] + k[8]*E*Sb_00/K[8] + k[14]*F*(1000.-Sb_00-Sb_01-Sb_10)/K[14]
    Sintg[5] = - k[13]*F*Sb_10/K[13] - k[11]*E*Sb_10/K[11] + k[9]*E*Sb_00/K[9] + k[15]*F*(1000.-Sb_00-Sb_01-Sb_10)/K[15]
             
    return(Sintg)

@jit
def check_convergence(v, trange, epsilon=1.0):
    """
    Judge if each state of a substrate is convergent.
    
    Parameters
    --------------
    v : array
    A sequence of a state of a substrate.
    trange : int
    The time the integration was done.
    epsilon : scalar
    A threshold for the judge.
    
    Returns
    ----------
    1 if not convergence.
    """
    
    rang = trange//10
    
    # check convergence
    vstd = np.std(v[-rang:])
    
    diffstd = np.std(np.diff(v[-rang:]))
    if diffstd < epsilon:
        return(3)
    elif vstd < epsilon:
        return(0)
    else:
        return(1) # not convergence
    
def bias(i_bias):
    """
    Specify which motif biases are posed between states of the substrates.
    
    Parameters
    --------------
    i_bias : int
    the index numbers of the biases that correspond to Figure 3.
    
    Returns
    ----------
    (Abias, Bbias) : tuple(list, list)
    bias position index for motif A and B.
    """
    
    # Motif A only
    if i_bias==1:
        return([[0], []])
    elif i_bias==2:
        return([[0, 1], []])
    elif i_bias==3:
        return([[0, 2], []])
    elif i_bias==4:
        return([[0, 3], []])
    elif i_bias==5:
        return([[0, 4], []])
    elif i_bias==6:
        return([[0, 3, 4], []])
    elif i_bias==7:
        return([[0, 3, 4, 7], []])
    
    # Motif B only
    elif i_bias==8:
        return([[], [0]])
    elif i_bias==9:
        return([[], [0, 1]])
    elif i_bias==10:
        return([[], [0, 2]])
    elif i_bias==11:
        return([[], [0, 3]])
    elif i_bias==12:
        return([[], [0, 4]])
    elif i_bias==13:
        return([[], [0, 3, 4]])
    elif i_bias==14:
        return([[], [0, 3, 4, 7]])
    
    # 1x motif A, 1x motif B
    elif i_bias==15:
        return([[0], [0]])
    elif i_bias==16:
        return([[0], [1]])
    elif i_bias==17:
        return([[0], [1]])
    elif i_bias==18:
        return([[0], [3]])
    elif i_bias==19:
        return([[0], [4]])

    # 2x motif A, 1x motif B
    elif i_bias==20:
        return([[0, 1], [3]])
    elif i_bias==21:
        return([[0, 1], [4]])
    elif i_bias==22:
        return([[0, 2], [1]])
    elif i_bias==23:
        return([[0, 2], [4]])
    elif i_bias==24:
        return([[0, 3], [2]])
    elif i_bias==25:
        return([[0, 3], [4]])
    elif i_bias==26:
        return([[0, 4], [1]])
    elif i_bias==27:
        return([[0, 4], [2]])
    elif i_bias==28:
        return([[0, 4], [3]])

    # 1x motif A, 2x motif B
    elif i_bias==29:
        return([[3], [0, 1]])
    elif i_bias==30:
        return([[4], [0, 1]])
    elif i_bias==31:
        return([[1], [0, 2]])
    elif i_bias==32:
        return([[4], [0, 2]])
    elif i_bias==33:
        return([[2], [0, 3]])
    elif i_bias==34:
        return([[4], [0, 3]])
    elif i_bias==35:
        return([[1], [0, 4]])
    elif i_bias==36:
        return([[2], [0, 4]])
    elif i_bias==37:
        return([[3], [0, 4]])
    
    # 2x motif A, 2x motif B
    elif i_bias==38:
        return([[0, 3], [0, 1]])
    elif i_bias==39:
        return([[0, 3], [0, 2]])
    elif i_bias==40:
        return([[0, 3], [0, 3]])
    elif i_bias==41:
        return([[0, 3], [0, 4]])
    elif i_bias==42:
        return([[0, 3], [1, 2]])
    elif i_bias==43:
        return([[0, 3], [1, 4]])
    elif i_bias==44:
        return([[0, 3], [4, 5]])
    elif i_bias==45:
        return([[0, 3], [4, 6]])
    elif i_bias==46:
        return([[0, 3], [4, 7]])
    
    
def bias2paramindex(bias):
    """
    specify which paramter should be biased given the bias position index.
    
    Paramters
    -------------
    bias : tuple(list, list)
    Abias and Bbias
    
    Returns
    ----------
    index_k : list
    indice of rate constants to be biased.
    index_K : list
    indice of MM constants to be biased.
    """
    
    index_k = []
    index_K = []
    Abias, Bbias = bias
    
    if not len(Abias)==0:
        for b in Abias: 
            if b==0:
                index_k += [4]
                index_K += [1]
            elif b==1:
                index_k += [5]
                index_K += [0]
            elif b==2:
                index_k += [2]
                index_K += [7]
            elif b==3:
                index_k += [3]
                index_K += [6]
            elif b==4:
                index_k += [12]
                index_K += [9]
            elif b==5:
                index_k += [13]
                index_K += [8]
            elif b==6:
                index_k += [10]
                index_K += [15]
            elif b==7:
                index_k += [11]
                index_K += [14]
    
    if not len(Bbias)==0:
        for b in Bbias:
            if b==0:
                index_k += [0]
                index_K += [4]
            elif b==1:
                index_k += [1]
                index_K += [5]
            elif b==2:
                index_k += [6]
                index_K += [2]
            elif b==3:
                index_k += [7]
                index_K += [3]
            elif b==4:
                index_k += [8]
                index_K += [12]
            elif b==5:
                index_k += [9]
                index_K += [13]
            elif b==6:
                index_k += [14]
                index_K += [10]
            elif b==7:
                index_k += [15]
                index_K += [11]
    return(index_k, index_K)
        
def gen_kK(index_k, index_K):
    """
    Randomly generate 32 parameters that determine the system.
    Then impose biases if specified by index_k or index_K.
    
    Paramters
    -------------
    index_k : list
    indice of rate constants to be biased.
    index_K : list
    indice of MM constants to be biased.
    
    Returns
    ----------
    k : array
    Rate constants.
    K : array
    MM constants.
    """
    
    rk = np.random.rand(16)
    rK = np.random.rand(16)
    
    k = 10**(3*rk)
    K = 10**(5*rK-2)
    
    k[index_k] = 10**(2+rk[index_k])
    K[index_K] = 10**(-2+rK[index_K])
    return(k, K)

def biased_search(args):
    """
    Iterate random parameter generation and classification of chaotic solutions.
    
    Parameters
    --------------
    args : tuple, shape (3)
        i_core : int
            Specify which cpu core is used.
        i_bias : int
            Specify which bias type is imposed.
        n_iter : int
            How much iteration is done by each cpu core.
    """
    
    i_core, i_bias, n_iter = args
    
    S0 = np.asarray([1000., 0., 0., 1000., 0., 0.]) # initial state of the substrates.  
    b = bias(i_bias)
    index_k, index_K = bias2paramindex(b)
    
    now = datetime.now()
    date = '{}_{}_{}_{}_{}_{}_{}'.format(now.year, now.month, now.day, now.hour, now.minute, now.second, i_core)
    np.random.seed(int('{}_{}_{}_{}_'.format(i_core, now.day, now.hour, now.minute)+str(now.microsecond)[-4:-2]))
    
    # the path to save the search results.
    filename = './random_{:02}.pickle'.format(i_core)
    #filename_osci = './random_{:02}_osci.pickle'.format(i_core)

    if os.path.isfile(filename):
        with open(filename, 'rb') as f:
            ongoing = pickle.load(f)
            i_iter = ongoing[0]
            n_osci = ongoing[1]
            chaos_maybe = ongoing[2:]
        #with open(filename_osci, 'rb') as f:
        #    osci_maybe = pickle.load(f)

    else:
        i_iter = np.zeros(1)
        n_osci = np.zeros(1)
        chaos_maybe = []
        #osci_maybe = []

    judge = np.zeros(6, dtype='int')

    st = time()
    for iteration in tqdm(range(int(n_iter))):
        i_iter += 1
        k, K = gen_kK(index_k, index_K)
        
        trange = 1000
        
        # quick integration
        S, info = odeint(func=DvD, y0=S0, t=np.arange(0, trange, 0.02), args=(k, K), atol=5.0e-4, rtol=5.0e-4, full_output=1)
        if error_message==info['message']:
            pass
        else:
            for col in range(6):
                judge[col] = check_convergence(v=S[:, col], trange=trange*50)
            if 1 in judge:
                # integration again
                S, info = odeint(func=DvD, y0=S0, t=np.arange(0, trange, 0.02), args=(k, K), full_output=1)
                if error_message==info['message']:
                    pass
                else:
                    for col in range(6):
                        judge[col] = check_convergence(v=S[:, col], trange=trange*50)
                    if 1 in judge:
                        trange = 6000
                        # careful integration
                        S, info = odeint(func=DvD, y0=S[-1, :], t=np.arange(0, trange, 0.02), args=(k,K), mxstep=10000, atol=1.0e-5, rtol=1.0e-5, full_output=1)
                        if error_message == info['message']:
                            pass
                        else:
                            for col in range(6):
                                judge[col] = check_convergence(v=S[:, col], trange=trange*50)
                            
                            # if not convergent, judge if oscillatory or chaotic. 
                            if 1 in judge:
                                f, Spw = periodogram(S[np.int(trange*50/2):], fs=50, axis=0)
                                maxfq_row = np.argmax(Spw)//Spw.shape[1]
                                maxfq_col = np.argmax(Spw)%Spw.shape[1]
                                maxfq_rate = np.sum(Spw[maxfq_row-2:maxfq_row+3, maxfq_col])/np.sum(Spw[:, maxfq_col])
                                if 0.15 > maxfq_rate:
                                    print('hit!')
                                    chaos_maybe.append([k, K]) # seems to be chaos 
                                else:
                                    n_osci += 1
                                    #osci_maybe.append([k, K]) # oscillation

        # save the intermediate result every hour.
        if (time()-st)>60*60:
            with open(filename, 'wb') as f:
                pickle.dump([i_iter, n_osci] + chaos_maybe, f)

            with open(filename_osci, 'wb') as f:
                pickle.dump(osci_maybe, f)

            st = time()

    print(filename)
    #with open(filename, 'wb') as f:
    #    pickle.dump([niter, nosci] + chaos_maybe, f)
    #with open(filename_osci, 'wb') as f:
    #    pickle.dump(osci_maybe, f)

    print(datetime.now(), 'Core{}: {} chaos_maybe pickled'.format(i_core, len(chaos_maybe)))
    #print(datetime.now(), 'Core{}: {} osci_maybe pickled'.format(i_core, len(osci_maybe)))
    
def multi_biased_search(n_cores, i_bias, n_iter_per_core=100000000):
    """
    A function to do biased search using multiple cpu cores.
    
    Parameters
    --------------
    n_cores : int
    How many cpu cores to use.
    i_bias : int
    Specify which bias type is imposed.
    n_iter_per_core : int
    How many iterations each core does.
    """
    
    args = []
    for i_core in range(n_cores):
        args.append((i_core, i_bias, n_iter_per_core))
    
    #print('Random search: using ' + str(cores) +' cores to explore chaos.')
    print('Random search: using {} cores to explore chaos.'.format(n_cores))
    with Pool(processes=n_cores) as pool:
        result = pool.map(biased_search, args)

In [ ]:
multi_DvD(10, 1, 10000)